# Hospital Facilities Insight 

Data obtained from Community Benefits Insight: http://www.communitybenefitinsight.org/api/get_hospitals.php

## Import Libaries

In [1]:
# import appropriate packages
import requests
import json 
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

import warnings
warnings.filterwarnings("ignore") #supress warnings

/var/folders/7x/mfyhhlkn0fz774bmn50_szsm0000gn/T/ipykernel_11866/3773660359.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Import Data

In [2]:
# API 
api_url = 'http://www.communitybenefitinsight.org/api/get_hospitals.php'

# request for api 
response = requests.get(api_url)

# check status
if response.status_code == 200:
    data = response.json()
    
    # data update check 
    last_modified_header = response.headers.get('Last-Modified')
    
    if last_modified_header:
        last_modified = datetime.strptime(last_modified_header, '%a, %d %b %Y %H:%M:%S GMT')
        print(f"Data was last modified: {last_modified}")

    # convert data to df 
    hospital_df = pd.DataFrame(data)
    
    print(hospital_df)
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")
    print(response.text)

     hospital_id hospital_org_id        ein  \
0              1               1  630307951   
1              2               2  630578923   
2              3               3  630312913   
3              4               4  630459034   
4              5               5  581973570   
...          ...             ...        ...   
3486        3487            2647  813040663   
3487        3488            2304  741109643   
3488        3489            2648  831954982   
3489        3490            2302  750800661   
3490        3491            2649  831869297   

                                     name  \
0                Mizell Memorial Hospital   
1                        St Vincents East   
2           Shelby Baptist Medical Center   
3            Callahan Eye Foundation Hosp   
4                 Cherokee Medical Center   
...                                   ...   
3486          Bsw Medical Center - Austin   
3487              Ascension Seton Bastrop   
3488         Texas Health Hosp

In [3]:
hospital_df # print df 

,hospital_id,hospital_org_id,ein,name,name_cr,street_address,city,state,zip_code,fips_state_and_county_code,hospital_bed_count,chrch_affl_f,urban_location_f,children_hospital_f,memb_counc_teach_hosps_f,medicare_provider_number,county,hospital_bed_size,updated_dt
0,1,1,630307951,Mizell Memorial Hospital,Mizell Memorial Hospital,702 Main Street,Opp,AL,36462,01039,99,N,N,N,N,010007,Covington County,<100 beds,"November 20, 2023"
1,2,2,630578923,St Vincents East,St Vincents East,50 Medical Park Drive East,Birmingham,AL,35235,01073,362,N,Y,N,Y,010011,Jefferson County,>299 beds,"November 20, 2023"
2,3,3,630312913,Shelby Baptist Medical Center,Shelby Baptist Medical Center,1000 First Street North,Alabaster,AL,35007,01117,252,N,Y,N,N,010016,Shelby County,100-299 beds,"November 20, 2023"
3,4,4,630459034,Callahan Eye Foundation Hosp,Callahan Eye Foundation Hosp,1720 University Boulevard,Birmingham,AL,35233,01073,106,N,Y,N,Y,010018,Jefferson County,100-299 beds,"November 20, 2023"
4,5,5,581973570,Cherokee Medical Center,Cherokee Medical Center,400 Northwood Drive,Centre,AL,35960,01019,60,N,N,N,N,010022,Cherokee County,<100 beds,"November 20, 2023"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3486,3487,2647,813040663,Bsw Medical Center - Austin,Bsw Medical Center - Austin,5245 W Us 290,Austin,TX,78735,48453,16,N,Y,N,N,670136,Travis County,<100 beds,"November 20, 2023"
3487,3488,2304,741109643,Ascension Seton Bastrop,Ascension Seton Bastrop,630 Highway 71 W,Bastrop,TX,78602,48021,7,N,Y,N,N,670143,Bastrop County,<100 beds,"November 20, 2023"
3488,3489,2648,831954982,Texas Health Hospital Frisco,Texas Health Hospital Frisco,12400 N Dallas Parkway,Frisco,TX,75033,48121,63,N,Y,N,N,670260,Denton County,<100 beds,"November 20, 2023"
3489,3490,2302,750800661,Methodist Midlothian Medical Center,Methodist Midlothian Medical Center,1201 E Highway 287,Midlothian,TX,76065,48139,46,N,Y,N,N,670300,Ellis County,<100 beds,"November 20, 2023"


In [4]:
# print 10 random samples from the dataframe 
hospital_df.sample(10)

,hospital_id,hospital_org_id,ein,name,name_cr,street_address,city,state,zip_code,fips_state_and_county_code,hospital_bed_count,chrch_affl_f,urban_location_f,children_hospital_f,memb_counc_teach_hosps_f,medicare_provider_number,county,hospital_bed_size,updated_dt
699,700,140,941105628,Maui Memorial Medical Center,Maui Memorial Medical Center,221 Mahalani Street,Wailuku,HI,96793,15009,213,N,Y,N,Y,120002,Maui County,100-299 beds,"November 20, 2023"
370,371,231,371705906,Llumc Murrieta,Llumc Murrieta,28062 Baxter Road,Murrieta,CA,92563,06065,106,N,Y,N,N,050770,Riverside County,100-299 beds,"November 20, 2023"
198,199,143,465322209,St Josephs Medical Center,St Josephs Medical Center,1800 N California Street,Stockton,CA,95204,06077,335,N,Y,N,Y,050084,San Joaquin County,>299 beds,"November 20, 2023"
3427,3428,2621,390806429,Aspirus Langlade Hospital,Aspirus Langlade Hospital,112 East Fifth Avenue,Antigo,WI,54409,55067,25,Y,N,N,Y,521350,Langlade County,<100 beds,"November 20, 2023"
3061,3062,2370,820551704,The Heart Hospital Baylor Denton,The Heart Hospital Baylor Denton,2801 South Mayhill,Denton,TX,76208,48121,16,N,Y,N,N,450893,Denton County,<100 beds,"November 20, 2023"
2511,2512,1989,260532635,Stroud Regional Medical Center,Stroud Regional Medical Center,2308 Highway 66 West,Stroud,OK,74079,40081,25,N,Y,N,N,371316,Lincoln County,<100 beds,"November 20, 2023"
3206,3207,2436,521988421,Rappahannock General Hospital,Rappahannock General Hospital,101 Harris Road,Kilmarnock,VA,22482,51103,76,N,N,N,N,491308,Lancaster County,<100 beds,"November 20, 2023"
1991,1992,1558,161533232,Degraff Memorial Hospital,,445 Tremont St,North Tonawanda,NY,14120,36063,155,N,Y,N,N,330007,Niagara County,100-299 beds,"November 20, 2023"
46,47,35,910939479,Ketchikan Medical Center,Ketchikan Medical Center,3100 Tongass Avenue,Ketchikan,AK,99901,02130,25,Y,N,N,N,021311,Ketchikan Gateway Borough,<100 beds,"November 20, 2023"
615,616,427,581911751,Sghs - Brunswick Campus,Sghs - Brunswick Campus,2415 Parkwood Drive,Brunswick,GA,31520,13127,300,N,Y,N,Y,110025,Glynn County,>299 beds,"November 20, 2023"


## Data Inspection

In [5]:
hospital_df.shape 

(3491, 19)

In [6]:
hospital_df.info() #all variables in df are labeled as object variables

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3491 entries, 0 to 3490
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   hospital_id                 3491 non-null   object
 1   hospital_org_id             3491 non-null   object
 2   ein                         3491 non-null   object
 3   name                        3491 non-null   object
 4   name_cr                     3491 non-null   object
 5   street_address              3491 non-null   object
 6   city                        3491 non-null   object
 7   state                       3491 non-null   object
 8   zip_code                    3491 non-null   object
 9   fips_state_and_county_code  3491 non-null   object
 10  hospital_bed_count          3491 non-null   object
 11  chrch_affl_f                3491 non-null   object
 12  urban_location_f            3491 non-null   object
 13  children_hospital_f         3491 non-null   obje

In [7]:
columns_to_convert = ['hospital_bed_count', 'medicare_provider_number']
binary_columns = ['chrch_affl_f', 'urban_location_f', 'children_hospital_f', 'memb_counc_teach_hosps_f']

# iterate through the list of columns
for col in columns_to_convert:
    # convert to numeric, coerce errors to NaN
    hospital_df[col] = pd.to_numeric(hospital_df[col], errors='coerce')
    
for col in binary_columns:
    # Map 'N' to 0 and 'Y' to 1
    hospital_df[col] = hospital_df[col].map({'N': 0, 'Y': 1})

# check the data types after conversion
hospital_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3491 entries, 0 to 3490
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   hospital_id                 3491 non-null   object
 1   hospital_org_id             3491 non-null   object
 2   ein                         3491 non-null   object
 3   name                        3491 non-null   object
 4   name_cr                     3491 non-null   object
 5   street_address              3491 non-null   object
 6   city                        3491 non-null   object
 7   state                       3491 non-null   object
 8   zip_code                    3491 non-null   object
 9   fips_state_and_county_code  3491 non-null   object
 10  hospital_bed_count          3491 non-null   int64 
 11  chrch_affl_f                3491 non-null   int64 
 12  urban_location_f            3491 non-null   int64 
 13  children_hospital_f         3491 non-null   int6

In [8]:
hospital_df.nunique() #unique variables 

hospital_id                   3491
hospital_org_id               2377
ein                           2377
name                          3330
name_cr                       3072
street_address                3439
city                          2192
state                           51
zip_code                      3200
fips_state_and_county_code    1751
hospital_bed_count             696
chrch_affl_f                     2
urban_location_f                 2
children_hospital_f              2
memb_counc_teach_hosps_f         2
medicare_provider_number      3491
county                        1195
hospital_bed_size                3
updated_dt                       1
dtype: int64

In [9]:
hospital_df.isnull().sum() #null values 

hospital_id                   0
hospital_org_id               0
ein                           0
name                          0
name_cr                       0
street_address                0
city                          0
state                         0
zip_code                      0
fips_state_and_county_code    0
hospital_bed_count            0
chrch_affl_f                  0
urban_location_f              0
children_hospital_f           0
memb_counc_teach_hosps_f      0
medicare_provider_number      0
county                        1
hospital_bed_size             0
updated_dt                    0
dtype: int64

In [10]:
duplicate_rows = hospital_df[hospital_df.duplicated()]

print("Duplicate Rows except first occurrence:")
print(duplicate_rows)

if hospital_df.duplicated().any():
    print("Duplicates exist in the DataFrame.")
else:
    print("No duplicates found in the DataFrame.")

Duplicate Rows except first occurrence:
Empty DataFrame
Columns: [hospital_id, hospital_org_id, ein, name, name_cr, street_address, city, state, zip_code, fips_state_and_county_code, hospital_bed_count, chrch_affl_f, urban_location_f, children_hospital_f, memb_counc_teach_hosps_f, medicare_provider_number, county, hospital_bed_size, updated_dt]
Index: []
No duplicates found in the DataFrame.


## Table Manipulation

In [28]:
# Hospital Information Table

# create unique ID for address
hospital_df['address_id'] = hospital_df['street_address'] + '_' + hospital_df['city'] + '_' + hospital_df['state'] + '_' + hospital_df['zip_code'] + '_' + hospital_df['fips_state_and_county_code']
hospital_df['address_id'] = hospital_df['address_id'].rank(method='dense', ascending=False).astype(int)

#create unique ID for hospital capacity
hospital_df['capacity_id'] = hospital_df['hospital_id'].astype(str) + '_' + hospital_df['hospital_bed_count'].astype(str) + '_' + hospital_df['hospital_bed_size']
hospital_df['capacity_id'] = hospital_df['capacity_id'].rank(method='dense', ascending=False).astype(int)


# create hospital information table
hospital = hospital_df[['hospital_id', 'hospital_org_id', 'name', 'ein', 'medicare_provider_number', 'address_id', 'capacity_id', 'updated_dt']]
hospital

,hospital_id,hospital_org_id,name,ein,medicare_provider_number,address_id,capacity_id,updated_dt
0,1,1,Mizell Memorial Hospital,630307951,10007,609,2381,"November 20, 2023"
1,2,2,St Vincents East,630578923,10011,1075,1270,"November 20, 2023"
2,3,3,Shelby Baptist Medical Center,630312913,10016,3323,667,"November 20, 2023"
3,4,4,Callahan Eye Foundation Hosp,630459034,10018,2440,556,"November 20, 2023"
4,5,5,Cherokee Medical Center,581973570,10022,1337,445,"November 20, 2023"
...,...,...,...,...,...,...,...,...
3486,3487,2647,Bsw Medical Center - Austin,813040663,670136,949,730,"November 20, 2023"
3487,3488,2304,Ascension Seton Bastrop,741109643,670143,718,729,"November 20, 2023"
3488,3489,2648,Texas Health Hospital Frisco,831954982,670260,2901,728,"November 20, 2023"
3489,3490,2302,Methodist Midlothian Medical Center,750800661,670300,2967,726,"November 20, 2023"


In [27]:
hospital_df

,hospital_id,hospital_org_id,ein,name,name_cr,street_address,city,state,zip_code,fips_state_and_county_code,...,chrch_affl_f,urban_location_f,children_hospital_f,memb_counc_teach_hosps_f,medicare_provider_number,county,hospital_bed_size,updated_dt,address_id,capacity_id
0,1,1,630307951,Mizell Memorial Hospital,Mizell Memorial Hospital,702 Main Street,Opp,AL,36462,01039,...,0,0,0,0,10007,Covington County,<100 beds,"November 20, 2023",609,2
1,2,2,630578923,St Vincents East,St Vincents East,50 Medical Park Drive East,Birmingham,AL,35235,01073,...,0,1,0,1,10011,Jefferson County,>299 beds,"November 20, 2023",1075,370
2,3,3,630312913,Shelby Baptist Medical Center,Shelby Baptist Medical Center,1000 First Street North,Alabaster,AL,35007,01117,...,0,1,0,0,10016,Shelby County,100-299 beds,"November 20, 2023",3323,490
3,4,4,630459034,Callahan Eye Foundation Hosp,Callahan Eye Foundation Hosp,1720 University Boulevard,Birmingham,AL,35233,01073,...,0,1,0,1,10018,Jefferson County,100-299 beds,"November 20, 2023",2440,680
4,5,5,581973570,Cherokee Medical Center,Cherokee Medical Center,400 Northwood Drive,Centre,AL,35960,01019,...,0,0,0,0,10022,Cherokee County,<100 beds,"November 20, 2023",1337,155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3486,3487,2647,813040663,Bsw Medical Center - Austin,Bsw Medical Center - Austin,5245 W Us 290,Austin,TX,78735,48453,...,0,1,0,0,670136,Travis County,<100 beds,"November 20, 2023",949,587
3487,3488,2304,741109643,Ascension Seton Bastrop,Ascension Seton Bastrop,630 Highway 71 W,Bastrop,TX,78602,48021,...,0,1,0,0,670143,Bastrop County,<100 beds,"November 20, 2023",718,63
3488,3489,2648,831954982,Texas Health Hospital Frisco,Texas Health Hospital Frisco,12400 N Dallas Parkway,Frisco,TX,75033,48121,...,0,1,0,0,670260,Denton County,<100 beds,"November 20, 2023",2901,135
3489,3490,2302,750800661,Methodist Midlothian Medical Center,Methodist Midlothian Medical Center,1201 E Highway 287,Midlothian,TX,76065,48139,...,0,1,0,0,670300,Ellis County,<100 beds,"November 20, 2023",2967,264


In [12]:
# Church Affiliation Table
church_affiliation = hospital_df[['hospital_id', 'chrch_affl_f', 'updated_dt']]
church_affiliation

,hospital_id,chrch_affl_f,updated_dt
0,1,0,"November 20, 2023"
1,2,0,"November 20, 2023"
2,3,0,"November 20, 2023"
3,4,0,"November 20, 2023"
4,5,0,"November 20, 2023"
...,...,...,...
3486,3487,0,"November 20, 2023"
3487,3488,0,"November 20, 2023"
3488,3489,0,"November 20, 2023"
3489,3490,0,"November 20, 2023"


In [29]:
# Hospital Capacity Table
hospital_capacity = hospital_df[['capacity_id', 'hospital_bed_size', 'hospital_bed_count', 'updated_dt']]
hospital_capacity

,capacity_id,hospital_bed_size,hospital_bed_count,updated_dt
0,2381,<100 beds,99,"November 20, 2023"
1,1270,>299 beds,362,"November 20, 2023"
2,667,100-299 beds,252,"November 20, 2023"
3,556,100-299 beds,106,"November 20, 2023"
4,445,<100 beds,60,"November 20, 2023"
...,...,...,...,...
3486,730,<100 beds,16,"November 20, 2023"
3487,729,<100 beds,7,"November 20, 2023"
3488,728,<100 beds,63,"November 20, 2023"
3489,726,<100 beds,46,"November 20, 2023"


In [14]:
# Children Hospital Table
children_hospital= hospital_df[['hospital_id', 'children_hospital_f', 'updated_dt']]
children_hospital

,hospital_id,children_hospital_f,updated_dt
0,1,0,"November 20, 2023"
1,2,0,"November 20, 2023"
2,3,0,"November 20, 2023"
3,4,0,"November 20, 2023"
4,5,0,"November 20, 2023"
...,...,...,...
3486,3487,0,"November 20, 2023"
3487,3488,0,"November 20, 2023"
3488,3489,0,"November 20, 2023"
3489,3490,0,"November 20, 2023"


In [15]:
# Teaching Table
teaching = hospital_df[['hospital_id', 'memb_counc_teach_hosps_f', 'updated_dt']]
teaching

,hospital_id,memb_counc_teach_hosps_f,updated_dt
0,1,0,"November 20, 2023"
1,2,1,"November 20, 2023"
2,3,0,"November 20, 2023"
3,4,1,"November 20, 2023"
4,5,0,"November 20, 2023"
...,...,...,...
3486,3487,0,"November 20, 2023"
3487,3488,0,"November 20, 2023"
3488,3489,0,"November 20, 2023"
3489,3490,0,"November 20, 2023"


In [16]:
# connect to database 
conn = sqlite3.connect('hospitalinsights.db')

In [30]:
# create Hospital Capacity Table with capacity_id as primary key
hospital_capacity.to_sql('HospitalCapacity', conn, if_exists='replace', index=False,
                         dtype={'capacity_id': 'PRIMARY KEY',
                                'hospital_bed_size': 'TEXT',
                                'hospital_bed_count': 'INTEGER',
                                'updated_dt': 'TEXT'}
                        )

3491

In [18]:
# create Children Hospital Table without a primary key
children_hospital.to_sql('ChildrenHospital', conn, if_exists='replace', index=False,
                         dtype={'hospital_id': 'INTEGER',
                                'children_hospital_f': 'INTEGER',
                                'updated_dt': 'TEXT'}
                        )

3491

In [19]:
# create Teaching Table without a primary key
teaching.to_sql('Teaching', conn, if_exists='replace', index=False,
                dtype={'hospital_id': 'INTEGER',
                       'memb_counc_teach_hosps_f': 'INTEGER',
                       'updated_dt': 'TEXT'}
               )

3491

In [20]:
cursor = conn.cursor()
cursor.execute('''PRAGMA foreign_keys=ON''')

# commit changes and close connection
conn.commit()
# conn.close()

print("Database created successfully")

Database created successfully


In [33]:
# testing databse 
try:
    # Execute a query to retrieve data from the 'Teaching' table
    cursor.execute("SELECT * FROM hospitalcapacity")

    # Fetch all rows from the result set
    rows = cursor.fetchall()

    # Print the retrieved data
    for row in rows:
        print(row)

except sqlite3.Error as e:
    print("Error executing SQL query:", e)

(2381, '<100 beds', 99, 'November 20, 2023')
(1270, '>299 beds', 362, 'November 20, 2023')
(667, '100-299 beds', 252, 'November 20, 2023')
(556, '100-299 beds', 106, 'November 20, 2023')
(445, '<100 beds', 60, 'November 20, 2023')
(334, '>299 beds', 344, 'November 20, 2023')
(223, '100-299 beds', 115, 'November 20, 2023')
(112, '<100 beds', 15, 'November 20, 2023')
(1, '<100 beds', 69, 'November 20, 2023')
(3381, '100-299 beds', 115, 'November 20, 2023')
(3270, '100-299 beds', 125, 'November 20, 2023')
(3159, '<100 beds', 57, 'November 20, 2023')
(3048, '<100 beds', 40, 'November 20, 2023')
(2937, '>299 beds', 409, 'November 20, 2023')
(2826, '<100 beds', 81, 'November 20, 2023')
(2715, '>299 beds', 323, 'November 20, 2023')
(2604, '<100 beds', 71, 'November 20, 2023')
(2493, '100-299 beds', 267, 'November 20, 2023')
(2382, '>299 beds', 349, 'November 20, 2023')
(2270, '100-299 beds', 162, 'November 20, 2023')
(2159, '100-299 beds', 122, 'November 20, 2023')
(2048, '>299 beds', 505, 'N